In [43]:
from neo4j import GraphDatabase
import requests
import os
from dotenv import load_dotenv
import pandas as pd

## Download 2024 B1G season from wikipedia

In [45]:
tables = pd.read_html('https://en.wikipedia.org/wiki/2024_Big_Ten_Conference_football_season')

In [ ]:
#Ignore playoff week where table only has two rows
week_tables =  [df for df in tables if ("Home team" in df.columns and df.shape[0] > 3)]

for week_table in week_tables:
    week_table['Home team'] = week_table['Home team'].str.replace(r'No\.\s*\d+\s+', '', regex=True)
    week_table['Visiting team'] = week_table['Visiting team'].str.replace(r'No\.\s*\d+\s+', '', regex=True)

In [73]:
schedule = pd.concat(week_tables)

In [80]:
conference_teams = ['Illinois',
 'Indiana',
 'Iowa',
 'Maryland',
 'Michigan',
 'Michigan State',
 'Minnesota',
 'Nebraska',
 'Northwestern',
 'Ohio State',
 'Oregon',
 'Penn State',
 'Purdue',
 'Rutgers',
 'UCLA',
 'USC',
 'Washington',
 'Wisconsin']
conference_schedule = schedule[(schedule['Home team'].isin(conference_teams))&(schedule['Visiting team'].isin(conference_teams))]

## Connect to Neo4j

In [81]:
load_dotenv()

uri = os.getenv('NEO4J_URI')
user = os.getenv('NEO4J_USER')
password = os.getenv('NEO4J_PASSWORD')
database = os.getenv('NEO4J_DATABASE')

In [82]:
driver = GraphDatabase.driver(uri, auth=(user, password), database=database)

## Load data to Neo4j

In [85]:
result = driver.execute_query("CREATE CONSTRAINT team_key IF NOT EXISTS FOR (t:Team) REQUIRE t.name IS NODE KEY")
print(result.summary.counters)

{'_contains_updates': True, 'constraints_added': 1}


In [86]:
load_query = """
    UNWIND $data AS row
    MERGE (h:Team {name:row.`Home team`})
    MERGE (a:Team {name:row.`Visiting team`})
    MERGE (h)-[:PLAYS]->(a)
    """

result = driver.execute_query(load_query, {"data": conference_schedule[['Home team', 'Visiting team']].to_dict("records")})
print(result.summary.counters)

{'_contains_updates': True, 'labels_added': 18, 'relationships_created': 81, 'nodes_created': 18, 'properties_set': 18}
